# Amazon Web scraper 
<img src='amazon.jpg' width=400>

###  Importing modules

In [6]:
import csv 
from bs4 import BeautifulSoup
# Firefox or chrome 
from selenium import webdriver

### Startup the webdriver

In [10]:
# Using chrome 
driver = webdriver.Chrome()
url = 'https://www.amazon.com/gp/help/customer/display.html?nodeId=202073580'
driver.get(url)

In [143]:
#create function to search particular products:

def get_val(some):
    url_val = 'https://www.amazon.com/s?k={}&crid=1AZQR15WRQJRI&sprefix=shirt+for+men%2Caps%2C446&ref=nb_sb_ss_ts-doa-p_1_13'
    some = some.replace(' ','+')
    return url_val.format(some)

### casual shirts search link

In [144]:
# Search something;
url = get_val('casual Shirts')
print("casual shirts search link :\n",url)

casual shirts search link :
 https://www.amazon.com/s?k=casual+Shirts&crid=1AZQR15WRQJRI&sprefix=shirt+for+men%2Caps%2C446&ref=nb_sb_ss_ts-doa-p_1_13


In [13]:
driver.get(url)

### Extract all datas

In [97]:
soup = BeautifulSoup(driver.page_source,'html.parser')
res = soup.find_all('div',{'data-component-type':'s-search-result'})
print("Total results: "+str(len(res)))

Total results: 60


### Prototype for datas

In [63]:
item = res[0]
#get link
atag = item.h2.a


In [ ]:
#find the item description:
descrip =atag.text.strip()
descrip

In [36]:
# Get the link of product
url_val2 = 'https://www.amazon.com/'+atag.get('href')

In [40]:
#Find the price of the product:
price_parent = item.find('span','a-price')

price = price_parent.find('span','a-offscreen').text

In [61]:
# Find the product ratings
rating = item.i.text
rating

'4.3 out of 5 stars'

In [59]:
# Find the review count:
review_count = item.find('span',{'class':'a-size-base'}).text
review_count

'359'

### Generalize pattern:

In [81]:
def extract_record(item):
    #Description and url
    atag = item.h2.a
    descrip =atag.text.strip()
    url_val2 = 'https://www.amazon.com/'+atag.get('href')
    try:
     # price:
        price_parent = item.find('span','a-price')
        price = price_parent.find('span','a-offscreen').text
    except AttributeError:
        return
    try:
        # ranting and reviewCount:
        rating = item.i.text
        review_count = item.find('span',{'class':'a-size-base'}).text
    except AttributeError:
        rating =''
        review_count = ''
    
    result = (descrip,price,rating,review_count,url_val2)
    return result

In [132]:
records = []
results = soup.find_all('div',{'data-component-type':'s-search-result'}) 

for item in results:
    record = extract_record(item)
    if record:
        records.append(record)

### View 1st record  data

In [133]:
records[0]

("Grace's Secret WomensTops Summer Top for Women Short Sleeve Casual Loose Tunic Top Shirts",
 '$18.99',
 '4.3 out of 5 stars',
 '359',
 'https://www.amazon.com//gp/slredirect/picassoRedirect.html/ref=pa_sp_atf_aps_sr_pg1_1?ie=UTF8&adId=A090332919KGPRHGT8U1T&url=%2FGraces-Secret-Womens-Summer-SealBrown%2Fdp%2FB094JBNX9X%2Fref%3Dsr_1_1_sspa%3Fcrid%3D1AZQR15WRQJRI%26dchild%3D1%26keywords%3Dcasual%2BShirts%26qid%3D1621928577%26sprefix%3Dshirt%2Bfor%2Bmen%252Caps%252C446%26sr%3D8-1-spons%26psc%3D1&qualifier=1621928577&id=6123135377136169&widgetName=sp_atf')

### Display the all price values:

In [134]:
prices = []
for i in records:
    prices.append(i[1])
prices

['$18.99',
 '$17.97',
 '$13.30',
 '$21.99',
 '$16.00',
 '$26.52',
 '$17.69',
 '$22.50',
 '$17.89',
 '$29.50',
 '$21.94',
 '$19.99',
 '$22.98',
 '$25.99',
 '$17.99',
 '$10.00',
 '$18.90',
 '$19.40',
 '$14.99',
 '$18.99',
 '$25.90',
 '$45.00',
 '$17.16',
 '$34.99',
 '$16.90',
 '$24.19',
 '$22.99',
 '$9.99',
 '$18.98',
 '$25.00',
 '$16.14',
 '$32.79',
 '$21.99',
 '$26.99',
 '$21.08',
 '$18.90',
 '$19.45',
 '$28.99',
 '$19.99',
 '$18.90',
 '$16.10',
 '$23.48',
 '$16.99',
 '$24.99',
 '$22.10',
 '$22.56',
 '$21.83',
 '$29.99',
 '$16.40',
 '$12.00',
 '$34.94',
 '$18.99',
 '$23.98',
 '$29.99',
 '$15.99',
 '$12.82']

### Store the records to csv file:

In [135]:

#Create csv file and store the datas:
with open('Records2.csv','w',newline='',encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Description','Prices','Rating','ReviewCount','URL'])
        writer.writerows(records)
    